### gseapy analysis of Velos_AW_20170831
gsea algorithm: http://software.broadinstitute.org/gsea/doc/GSEAUserGuideFrame.html  
gseapy: https://media.readthedocs.org/pdf/gseapy/latest/gseapy.pdf  


In [1]:
import sys
import os
from itertools import chain
from collections import defaultdict
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('precision', 3)
pd.set_option('display.max_colwidth', -1)
import gseapy as gp
import goatools

obodag = goatools.obo_parser.GODag('go-basic.obo')

load obo file go-basic.obo
go-basic.obo: fmt(1.2) rel(2018-01-04) 49,283 GO Terms


In [2]:
sys.path.insert(0, "/home/gstupp/projects/metaproteomics")
from metaproteomics import utils
#from metaproteomics.analysis import build_loci

BASE = 'out/'
grouped_loci = utils.load(os.path.join(BASE,"grouped_loci_filt_annot.pkl.gz"))

In [3]:
def make_go2Gene_map(grouped_loci, ontology='MF'):    

    ontology_map = {'MF': 'molecular_function', 'BP': 'biological_process', 'CC': 'cellular_component'}
    
    out = defaultdict(set)    
    for l in grouped_loci:
        if 'go' in l.annotations:
            for go in l.annotations['go']:
                if obodag[go].namespace == ontology_map[ontology]:
                    out[go].add(l.cluster_id)
                    for parent in obodag[go].get_all_parents():
                        if obodag[parent].namespace == ontology_map[ontology]:
                            out[parent].add(l.cluster_id)
                
    return dict(out)

def filter_go2gene_map(go_locus):
    
    # Remove "very broad" gene sets. Arbitrary definition: gene sets that emcompass >50% of all IDs
    all_ids = set(chain(*go_locus.values()))
    go_locus = {key: value for (key, value) in go_locus.items() if len(value) / len(all_ids) <= 0.5}

    # Remove terms with less than 5 members: changed from 10 to 5 becasue small #s of proteins compared
    # to what you would find wiht genes
    go_locus = {key: value for (key, value) in go_locus.items() if len(value) >= 5}

    # Remove child terms with identical gene sets as their parents
    to_remove = set()
    for parent in go_locus.keys():
        # If child term has exact same members as parent, remove
        child_ids = [x.id for x in obodag[parent].children if x.id in go_locus.keys()]
        for child in child_ids:
            if go_locus[child] == go_locus[parent]:
                to_remove.add(child)
    go_locus = {key: value for (key, value) in go_locus.items() if key not in to_remove}

    # Remove sibling terms with identical gene sets
    to_remove = set()
    to_keep = set()
    for brother in go_locus.keys():
        to_keep.add(brother) # make sure filtered out siblings don't filter out ones we want to keep
        for parent in obodag[brother].parents:
            siblings = set([y.id for y in parent.children])
            if brother in siblings:
                siblings.remove(brother)
            for sibling in siblings:
                if sibling in go_locus.keys() and go_locus[brother] == go_locus[sibling] and not sibling in to_keep:
                    to_remove.add(sibling)
    go_locus = {key: value for (key, value) in go_locus.items() if key not in to_remove}

    return go_locus

def gomap_to_csv(go2gene, out_file = 'test.tsv'):

    out = ""
    for term, loci in go2gene.items():
        out += "{}\t".format(term)
        out += "{}\t".format(obodag[term].name)
        out += '\t'.join(list(map(str,loci)))
        out += '\n'
        
    with open(out_file, 'w') as fout:
        fout.write(out)

In [4]:
def run_go_gsea(rank_df, g2g_map, seed, outdir='tmp'):
    """
    A ranked df and go2gene mapping returns the result dataframe for GSEA against all go-Terms
    
    loci must be grouped such that avg_ratio and p-values are correct for 1 phenotype
    see rt_unenr_grouped_loci above for example
    """
    import gseapy as gp
        
    # save the go 2 gene map, since gseapy doesn't seem to be able to use one already in memory
    gomap_to_csv(g2g_map, 'temp.gmt')
    
    res = gp.prerank(rnk=rank_df, gene_sets='temp.gmt', outdir=outdir, min_size = 5, max_size=500, 
                     permutation_n = 10000, graph_num = 20, seed=seed)
      
    def get_go_name(term):
        return obodag[term].name
    
    res['name'] = res.index.map(get_go_name)
    
    return res.sort_values('nes', ascending=False)

def plot_gsea_result(row, rank):
    return gp.plot.gsea_plot(rank, row['name'], row.hit_index, row.nes, row.pval, row.fdr, row.rank_ES, phenoPos='Tcell', phenoNeg='RAG')

In [5]:
mf_map = make_go2Gene_map(grouped_loci)
mf_map_f = filter_go2gene_map(mf_map)

print('Unfiltered: {}\tFiltered: {}'.format(len(mf_map), len(mf_map_f)))

bp_map = make_go2Gene_map(grouped_loci, 'BP')
bp_map_f = filter_go2gene_map(bp_map)

print('Unfiltered: {}\tFiltered: {}'.format(len(bp_map), len(bp_map_f)))

cc_map = make_go2Gene_map(grouped_loci, 'CC')
cc_map_f = filter_go2gene_map(cc_map)

print('Unfiltered: {}\tFiltered: {}'.format(len(cc_map), len(cc_map_f)))

Unfiltered: 647	Filtered: 329
Unfiltered: 697	Filtered: 320
Unfiltered: 92	Filtered: 51


### probe_unenriched_gsea

In [6]:
out_dir = "probe_unenriched_gsea"
df = pd.read_csv(os.path.join(BASE,"probe_unenriched_deseq_results.csv"))
df = df[(df.padj.abs()<=0.2)]
df['log2FoldChange'] = -1 * df['log2FoldChange']
rank_df = df[['Unnamed: 0', 'log2FoldChange']].rename(columns={'Unnamed: 0': 'gene_name', 'log2FoldChange': 'rank'})
rank_df = rank_df.sort_values('rank').reset_index(drop=True)

In [7]:
rank_df.head()

,gene_name,rank
0,22471287,-9.155
1,17773601,-8.763
2,13554466,-8.646
3,23967352,-8.433
4,80889105,-8.278


In [8]:
mf_res = run_go_gsea(rank_df, mf_map_f, seed=1111, outdir=out_dir)
bp_res = run_go_gsea(rank_df, bp_map_f, seed=1111, outdir=out_dir)
cc_res = run_go_gsea(rank_df, cc_map_f, seed=1111, outdir=out_dir)

2018-04-02 11:45:11,780 Parsing data files for GSEA.............................
/home/gstupp/.local/lib/python3.5/site-packages/numpy/lib/arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
2018-04-02 11:45:14,265 0125 gene_sets used for further statistical testing.....
2018-04-02 11:45:14,266 Start to run GSEA...Might take a while..................
2018-04-02 12:01:34,050 Start to generate gseapy reports, and produce figures...
/usr/lib/python3/dist-packages/matplotlib/figure.py:1744: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "
2018-04-02 12:01:42,386 Congratulations...GSEAPY run successfully...............
2018-04-02 12:01:42,430 Parsing data files for GSEA.............................
2018-04-02 12:01:44,881 0142 gene_sets used for further 

In [17]:
mf_rt = mf_res.query('nes > 0 and pval < 0.01').sort_values('nes', ascending=False)
mf_control = mf_res.query('nes < 0 and pval < 0.01').sort_values('nes', ascending=True)
bp_rt = bp_res.query('nes > 0 and pval < 0.01').sort_values('nes', ascending=False)
bp_control = bp_res.query('nes < 0 and pval < 0.01').sort_values('nes', ascending=True)
cc_rt = cc_res.query('nes > 0 and pval < 0.01').sort_values('nes', ascending=False)
cc_control = cc_res.query('nes < 0 and pval < 0.01').sort_values('nes', ascending=True)

In [18]:
# rt is increased in VK samples == positive FC

In [20]:
mf_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0016875,0.613,3.983,0.000e+00,0.000e+00,474,173,"[58741189, 38897143, 44897813, 61990466, 62068470, 29514937, 63039990, 65015920, 43649735, 50478223, 53019891, 46879023, 64504429, 45642681, 48894783, 46572451, 49731876, 51370435, 37387111, 51733624, 55761561, 37246072, 64685086, 61125530, 63337935, 37614877, 37019022, 36257473, 40311155, 62319339, 62981654, 48821943, 15299206, 64049087, 57523511, 64262763, 57143423, 40038469, 46254344, 65426978, 40075163, 62451768, 63028596, 32703678, 61655230, 21296790, 36263384, 43895742, 65917826, 21265713, 50962173, 63135109, 63349006, 62636169, 44293582, 59648286, 62302959, 51551442, 51315228, 47089758, 45190111, 57966590, 61615729, 38920005, 20636313, 61692759, 36577121, 20753465, 62351383, 61665189, 52065007, 64131792, 11530933, 61118049, 59471295, 64124204, 65110127, 62033484, 61672663, 33335682, 61690012, 49276873, 56237374, 60315565, 61608599, 45744253, 61969939, 64139886, 50421186, 59124828, 39411364, 37277974, 62274242, 40538371, 37229072, 51901930, 37673633, 43930976, 46528790, 62818676, ...]","ligase activity, forming carbon-oxygen bonds"
GO:0140101,0.613,3.964,0.000e+00,0.000e+00,475,173,"[58741189, 38897143, 44897813, 61990466, 62068470, 29514937, 63039990, 65015920, 43649735, 50478223, 53019891, 46879023, 64504429, 45642681, 48894783, 46572451, 49731876, 51370435, 37387111, 51733624, 55761561, 37246072, 64685086, 61125530, 63337935, 37614877, 37019022, 36257473, 40311155, 62319339, 62981654, 48821943, 15299206, 64049087, 57523511, 64262763, 57143423, 40038469, 46254344, 65426978, 40075163, 62451768, 63028596, 32703678, 61655230, 21296790, 36263384, 43895742, 65917826, 21265713, 50962173, 63135109, 63349006, 62636169, 44293582, 59648286, 62302959, 51551442, 51315228, 47089758, 45190111, 57966590, 61615729, 38920005, 20636313, 61692759, 36577121, 20753465, 62351383, 61665189, 52065007, 64131792, 11530933, 61118049, 59471295, 64124204, 65110127, 62033484, 61672663, 33335682, 61690012, 49276873, 56237374, 60315565, 61608599, 45744253, 61969939, 64139886, 50421186, 59124828, 39411364, 37277974, 62274242, 40538371, 37229072, 51901930, 37673633, 43930976, 46528790, 62818676, ...]","catalytic activity, acting on a tRNA"
GO:0004812,0.622,3.963,0.000e+00,0.000e+00,468,169,"[58741189, 38897143, 44897813, 61990466, 62068470, 29514937, 63039990, 65015920, 43649735, 50478223, 53019891, 46879023, 64504429, 45642681, 48894783, 46572451, 49731876, 51370435, 37387111, 51733624, 55761561, 37246072, 63337935, 37614877, 37019022, 36257473, 40311155, 62319339, 62981654, 48821943, 15299206, 64049087, 57523511, 64262763, 57143423, 40038469, 46254344, 65426978, 40075163, 62451768, 63028596, 32703678, 61655230, 21296790, 36263384, 43895742, 65917826, 21265713, 50962173, 63135109, 63349006, 62636169, 44293582, 59648286, 62302959, 51551442, 51315228, 47089758, 45190111, 57966590, 61615729, 38920005, 20636313, 61692759, 36577121, 20753465, 62351383, 61665189, 52065007, 64131792, 11530933, 61118049, 59471295, 64124204, 65110127, 62033484, 61672663, 33335682, 61690012, 49276873, 56237374, 60315565, 61608599, 45744253, 61969939, 64139886, 50421186, 59124828, 39411364, 37277974, 62274242, 40538371, 37229072, 51901930, 37673633, 43930976, 46528790, 62818676, 55842052, 58900425, ...]",aminoacyl-tRNA ligase activity
GO:0015078,0.664,3.635,0.000e+00,0.000e+00,115,70,"[62200140, 4557438, 80333174, 38309702, 59671387, 44272259, 47779625, 66164952, 37757410, 16294767, 44878788, 38747542, 42598420, 18663560, 10517020, 50441137, 6211961, 7298112, 18068175, 13225945, 8043288, 10920382, 18219005, 32363526, 17566661, 62623184, 64874050, 60728781, 4763573, 20853216, 62543608, 66340436, 62501359, 14739479, 20819950, 20976235, 18669105, 5377188, 44007614, 66130141, 46925916, 38471456, 32313561, 46694653, 37477462, 18263576, 7438775, 2287574, 166464249, 12867643, 62398150, 21049052, 1809293, 7089159, 7441750, 4137138, 62521088, 61472524, 21076498, 1210665

In [21]:
mf_control

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0004372,-0.735,-4.274,0.000e+00,0.000,361,342,"[54043877, 39548280, 37667489, 46516391, 58291687, 8099862, 62813986, 62917179, 62579411, 69107994, 69040461, 68470812, 68342942, 68305253, 67628267, 67269704, 66921778, 10932283, 66877191, 66832817, 66776973, 66768011, 66545558, 67463692, 69236623, 69244066, 69299112, 167690204, 167672220, 166640109, 166268290, 166074821, 165852677, 168235116, 79205226, 76162082, 74452649, 69764132, 69571229, 69554462, 69454839, 80395477, 70321612, 10941645, 9700083, 8057526, 12904533, 8195337, 8615293, 8644821, 44039083, 8760323, 42061238, 41668931, 11103382, 41381210, 8888620, 40923461, 9628878, 8957519, 44278217, 44869523, 10899788, 62848646, 62849225, 62851043, 7963239, 45992970, 12604573, 45817127, 12245588, 12271762, 12380804, 8043328, 45138987, 21887746, 40788256, 39335625, 62886393, 62887343, 59309002, 21254768, 59123773, 62864184, 13146585, 13394096, 5224748, 5465314, 9056874, 5562437, 13202926, 62852705, 21344859, 5165637, 60275234, 9052340, 21850717, 21834008, 39651603, 21590822, ...]",glycine hydroxymethyltransferase activity
GO:0016742,-0.695,-4.056,0.000e+00,0.000,418,356,"[62260224, 55980982, 36324260, 41357783, 39205302, 13441941, 54043877, 39548280, 37667489, 46516391, 4096855, 58291687, 166408448, 63471040, 63063318, 8173354, 8099862, 167629757, 59141579, 62813986, 62917179, 63263599, 62579411, 69107994, 69040461, 68470812, 68342942, 68305253, 67628267, 67269704, 66921778, 10932283, 66877191, 66832817, 66776973, 66768011, 66545558, 67463692, 69236623, 69244066, 69299112, 167690204, 167672220, 166640109, 166268290, 166074821, 165852677, 168235116, 79205226, 76162082, 74452649, 69764132, 69571229, 69554462, 69454839, 80395477, 70321612, 10941645, 9700083, 8057526, 12904533, 8195337, 8615293, 8644821, 44039083, 8760323, 42061238, 41668931, 11103382, 41381210, 8888620, 40923461, 9628878, 8957519, 44278217, 44869523, 10899788, 62848646, 62849225, 62851043, 7963239, 45992970, 12604573, 45817127, 12245588, 12271762, 12380804, 8043328, 45138987, 21887746, 40788256, 39335625, 62886393, 62887343, 59309002, 21254768, 59123773, 62864184, 13146585, 13394096, ...]","hydroxymethyl-, formyl- and related transferase activity"
GO:0016741,-0.663,-3.890,0.000e+00,0.000,455,376,"[62260224, 47954303, 64217941, 55980982, 36324260, 41357783, 39205302, 13441941, 54043877, 33637358, 37130540, 47085131, 63362769, 54312602, 66934825, 47226047, 51474648, 20122920, 46883859, 66406374, 41409343, 39548280, 37667489, 46516391, 4096855, 58291687, 166408448, 63471040, 63063318, 8173354, 8099862, 167629757, 59141579, 62813986, 62917179, 63263599, 62579411, 63304867, 21396601, 69107994, 69040461, 68470812, 68342942, 68305253, 67628267, 67269704, 66921778, 10932283, 66877191, 66832817, 66776973, 66768011, 66545558, 67463692, 69236623, 69244066, 69299112, 167690204, 167672220, 166640109, 166268290, 166074821, 165852677, 168235116, 79205226, 76162082, 74452649, 69764132, 69571229, 69554462, 69454839, 80395477, 70321612, 10941645, 9700083, 8057526, 12904533, 8195337, 8615293, 8644821, 44039083, 8760323, 42061238, 41668931, 11103382, 41381210, 8888620, 40923461, 9628878, 8957519, 44278217, 44869523, 10899788, 62848646, 62849225, 62851043, 7963239, 45992970, 12604573, 45817127, ...]","transferase activity, transferring one-carbon groups"
GO:0004872,-0.683,-3.626,0.000e+00,0.000,170,158,"[49582748, 38737848, 52808333, 43266065, 44499302, 43424619, 52609475, 59052976, 42005004, 21008359, 43082532, 36220708, 50420785, 52863741, 57209297, 54098061, 61616124, 36293304, 51540385, 61632074, 61649530, 63844951, 58536749, 55774214, 56581658, 61638635, 39489881, 62395108, 61569703, 52655657, 59055758, 52061002, 44079911, 38310319, 51323646, 42758340, 42827533, 55769380, 61656451, 52418189, 44570491, 61654976, 65953578, 37416311, 55935267, 61657168, 38135581, 20940876, 42570082, 37457633, 58552473, 36811313, 61637893, 11697747, 39961135, 57221749, 43782161, 6170

In [22]:
bp_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0006101,0.875,4.303,0.000e+00,0.000e+00,50,45,"[80889105, 2114380, 77369443, 13684061, 4737878, 74423985, 23757178, 12347054, 3740901, 60432088, 26000455, 11513369, 17037782, 21164047, 45212217, 17586159, 80527010, 6662432, 13649322, 26904631, 13294338, 18316344, 61830050, 14103780, 33672600, 5863982, 16897740, 33675780, 18065556, 8650619, 18285157, 13248141, 25953698, 65679484, 78477008, 13649321, 61839169, 29983272, 29001612, 5484467, 7496609, 10697047, 20853009, 14670524, 22703971]",citrate metabolic process
GO:0072350,0.875,4.301,0.000e+00,0.000e+00,60,45,"[80889105, 2114380, 77369443, 13684061, 4737878, 74423985, 23757178, 12347054, 3740901, 60432088, 26000455, 11513369, 17037782, 21164047, 45212217, 17586159, 80527010, 6662432, 13649322, 26904631, 13294338, 18316344, 61830050, 14103780, 33672600, 5863982, 16897740, 33675780, 18065556, 8650619, 18285157, 13248141, 25953698, 65679484, 78477008, 13649321, 61839169, 29983272, 29001612, 5484467, 7496609, 10697047, 20853009, 14670524, 22703971]",tricarboxylic acid metabolic process
GO:0006418,0.618,3.949,0.000e+00,0.000e+00,465,165,"[58741189, 38897143, 44897813, 61990466, 62068470, 29514937, 63039990, 65015920, 43649735, 50478223, 53019891, 46879023, 64504429, 45642681, 48894783, 46572451, 49731876, 51370435, 37387111, 51733624, 55761561, 37246072, 63337935, 37614877, 37019022, 36257473, 40311155, 62319339, 62981654, 48821943, 15299206, 64049087, 57523511, 64262763, 57143423, 40038469, 46254344, 65426978, 40075163, 62451768, 63028596, 32703678, 61655230, 21296790, 36263384, 43895742, 65917826, 21265713, 50962173, 63135109, 63349006, 62636169, 59648286, 62302959, 51551442, 51315228, 47089758, 45190111, 57966590, 61615729, 38920005, 20636313, 61692759, 36577121, 20753465, 62351383, 61665189, 52065007, 64131792, 11530933, 61118049, 64124204, 65110127, 62033484, 61672663, 33335682, 61690012, 49276873, 56237374, 60315565, 61608599, 45744253, 61969939, 64139886, 50421186, 39411364, 37277974, 62274242, 40538371, 51901930, 37673633, 43930976, 46528790, 62818676, 55842052, 58900425, 43251581, 65072096, 66036718, 65127974, ...]",tRNA aminoacylation for protein translation
GO:0043038,0.609,3.914,0.000e+00,0.000e+00,471,169,"[58741189, 38897143, 44897813, 61990466, 62068470, 29514937, 63039990, 65015920, 43649735, 50478223, 53019891, 46879023, 64504429, 45642681, 48894783, 46572451, 49731876, 51370435, 37387111, 51733624, 55761561, 37246072, 64685086, 61125530, 63337935, 37614877, 37019022, 36257473, 40311155, 62319339, 62981654, 48821943, 15299206, 64049087, 57523511, 64262763, 57143423, 40038469, 46254344, 65426978, 40075163, 62451768, 63028596, 32703678, 61655230, 21296790, 36263384, 43895742, 65917826, 21265713, 50962173, 63135109, 63349006, 62636169, 59648286, 62302959, 51551442, 51315228, 47089758, 45190111, 57966590, 61615729, 38920005, 20636313, 61692759, 36577121, 20753465, 62351383, 61665189, 52065007, 64131792, 11530933, 61118049, 64124204, 65110127, 62033484, 61672663, 33335682, 61690012, 49276873, 56237374, 60315565, 61608599, 45744253, 61969939, 64139886, 50421186, 39411364, 37277974, 62274242, 40538371, 51901930, 37673633, 43930976, 46528790, 62818676, 55842052, 58900425, 43251581, 65072096, ...]",amino acid activation
GO:0034660,0.584,3.776,0.000e+00,0.000e+00,478,174,"[58741189, 38897143, 44897813, 61990466, 62068470, 29514937, 63039990, 65015920, 43649735, 50478223, 53019891, 46879023, 64504429, 45642681, 48894783, 46572451, 49731876, 51370435, 37387111, 51733624, 55761561, 37246072, 64685086, 61125530, 63337935, 37614877, 37019022, 36257473, 40311155, 62319339, 62981654, 54730221, 48821943, 15299206, 64049087, 57523511, 64262763, 57143423, 40038469, 46254344, 65426978, 40075163, 62451768, 63028596, 32703678, 61655230, 21296790, 36263384, 43895742, 65917826, 21265713, 50962173, 63135109, 63349006, 62636169, 59648286, 62302959, 51551442, 51315228, 47089758, 45190111, 57966590, 61615729, 38920005, 2063

In [23]:
bp_control

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0006563,-0.715,-4.175,0.000e+00,0.000e+00,378,354,"[18125231, 22853286, 9760084, 54043877, 5165836, 67600895, 39548280, 36975417, 37667489, 46516391, 20718571, 61888884, 58291687, 8099862, 62813986, 62917179, 62579411, 69107994, 69040461, 68470812, 68342942, 68305253, 67628267, 67269704, 66921778, 10932283, 66877191, 66832817, 66776973, 66768011, 66545558, 67463692, 69236623, 69244066, 69299112, 167690204, 167672220, 166640109, 166268290, 166074821, 165852677, 168235116, 79205226, 76162082, 74452649, 69764132, 69571229, 69554462, 69454839, 80395477, 70321612, 10941645, 9700083, 8057526, 12904533, 8195337, 8615293, 8644821, 44039083, 8760323, 42061238, 41668931, 11103382, 41381210, 8888620, 40923461, 9628878, 8957519, 44278217, 44869523, 10899788, 62848646, 62849225, 62851043, 7963239, 45992970, 12604573, 45817127, 12245588, 12271762, 12380804, 8043328, 45138987, 21887746, 40788256, 39335625, 62886393, 62887343, 59309002, 21254768, 59123773, 62864184, 13146585, 13394096, 5224748, 5465314, 9056874, 5562437, 13202926, 62852705, ...]",L-serine metabolic process
GO:0006544,-0.714,-4.174,0.000e+00,0.000e+00,370,350,"[61678153, 47954303, 43685204, 54043877, 37130540, 63362769, 51474648, 20122920, 39548280, 37667489, 46516391, 58291687, 8099862, 62813986, 62917179, 62579411, 69107994, 69040461, 68470812, 68342942, 68305253, 67628267, 67269704, 66921778, 10932283, 66877191, 66832817, 66776973, 66768011, 66545558, 67463692, 69236623, 69244066, 69299112, 167690204, 167672220, 166640109, 166268290, 166074821, 165852677, 168235116, 79205226, 76162082, 74452649, 69764132, 69571229, 69554462, 69454839, 80395477, 70321612, 10941645, 9700083, 8057526, 12904533, 8195337, 8615293, 8644821, 44039083, 8760323, 42061238, 41668931, 11103382, 41381210, 8888620, 40923461, 9628878, 8957519, 44278217, 44869523, 10899788, 62848646, 62849225, 62851043, 7963239, 45992970, 12604573, 45817127, 12245588, 12271762, 12380804, 8043328, 45138987, 21887746, 40788256, 39335625, 62886393, 62887343, 59309002, 21254768, 59123773, 62864184, 13146585, 13394096, 5224748, 5465314, 9056874, 5562437, 13202926, 62852705, 21344859, ...]",glycine metabolic process
GO:0001505,-0.714,-4.158,0.000e+00,0.000e+00,372,350,"[61678153, 47954303, 43685204, 54043877, 37130540, 63362769, 51474648, 20122920, 39548280, 37667489, 46516391, 58291687, 8099862, 62813986, 62917179, 62579411, 69107994, 69040461, 68470812, 68342942, 68305253, 67628267, 67269704, 66921778, 10932283, 66877191, 66832817, 66776973, 66768011, 66545558, 67463692, 69236623, 69244066, 69299112, 167690204, 167672220, 166640109, 166268290, 166074821, 165852677, 168235116, 79205226, 76162082, 74452649, 69764132, 69571229, 69554462, 69454839, 80395477, 70321612, 10941645, 9700083, 8057526, 12904533, 8195337, 8615293, 8644821, 44039083, 8760323, 42061238, 41668931, 11103382, 41381210, 8888620, 40923461, 9628878, 8957519, 44278217, 44869523, 10899788, 62848646, 62849225, 62851043, 7963239, 45992970, 12604573, 45817127, 12245588, 12271762, 12380804, 8043328, 45138987, 21887746, 40788256, 39335625, 62886393, 62887343, 59309002, 21254768, 59123773, 62864184, 13146585, 13394096, 5224748, 5465314, 9056874, 5562437, 13202926, 62852705, 21344859, ...]",regulation of neurotransmitter levels
GO:0009069,-0.696,-4.064,0.000e+00,0.000e+00,387,362,"[61678153, 18125231, 22853286, 47954303, 43685204, 9760084, 54043877, 5165836, 37130540, 67600895, 63362769, 51474648, 20122920, 39548280, 36975417, 37667489, 46516391, 20718571, 61888884, 58291687, 8099862, 62813986, 62917179, 62579411, 69107994, 69040461, 68470812, 68342942, 68305253, 67628267, 67269704, 66921778, 10932283, 66877191, 66832817, 66776973, 66768011, 66545558, 67463692, 69236623, 69244066, 69299112, 167690204, 167672220, 166640109, 166268290, 166074821, 165852677, 168235116, 79205226, 76162082, 74452649, 69764132, 69571229, 69554462, 69454839, 80395477, 70321612, 10941645, 9700083, 8057526, 12904533, 8195337, 8

In [24]:
cc_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0033178,0.663,3.514,0.000,0.000,119,61,"[62200140, 4557438, 62156405, 80333174, 38309702, 59671387, 66164952, 37757410, 16294767, 44878788, 38747542, 42598420, 18663560, 10517020, 6211961, 7298112, 13225945, 8043288, 10920382, 18219005, 32363526, 17566661, 62623184, 4763573, 20853216, 62543608, 62501359, 14739479, 20976235, 18669105, 5377188, 32313561, 18263576, 7438775, 166464249, 12867643, 62398150, 21049052, 1809293, 7089159, 7441750, 4137138, 62810060, 62521088, 21076498, 168069660, 166958817, 65122773, 62422194, 51220241, 9191633, 62095026, 12106659, 62354250, 40132861, 62665934, 9505311, 62694751, 63423769, 59732024, 62659413]","proton-transporting two-sector ATPase complex, catalytic domain"
GO:0045261,0.777,3.109,0.000,0.000,41,22,"[62200140, 4557438, 80333174, 38309702, 59671387, 66164952, 37757410, 16294767, 44878788, 38747542, 42598420, 10517020, 6211961, 7298112, 13225945, 14739479, 32313561, 12867643, 62398150, 21049052, 1809293, 40132861]","proton-transporting ATP synthase complex, catalytic core F(1)"
GO:0043234,0.309,1.882,0.000,0.003,194,121,"[62200140, 4557438, 62156405, 63672434, 80333174, 38309702, 59671387, 66164952, 37757410, 16294767, 44878788, 38747542, 42598420, 18663560, 64976569, 10517020, 6211961, 7298112, 13225945, 8043288, 10920382, 18219005, 32363526, 17566661, 62623184, 4763573, 20853216, 62543608, 167585308, 62501359, 14739479, 20976235, 18669105, 5377188, 53308075, 41809447, 32313561, 18263576, 7438775, 166464249, 12867643, 62398150, 167286618, 21049052, 62861628, 1809293, 7089159, 7441750, 4137138, 62810060, 62521088, 21076498, 168069660, 166958817, 65122773, 62422194, 51220241, 9191633, 62095026, 12106659, 62354250, 62704545, 40132861, 62665934, 40686252, 9505311, 62694751, 63332505, 63423769, 59732024, 62659413, 62830891, 63018596, 62883525, 63133221, 61865720, 165644696, 20629788, 29371220, 57584638, 5711881, 63452797, 63468226, 51679236, 62545344, 8187004, 63078357, 166608363, 28240379, 58453185, 63132923, 77351967, 28669498, 28360306, 15700990, 17587826, 66373416, 6569127, 28273397, 20710384, ...]",protein complex
GO:0098796,0.300,1.780,0.000,0.005,171,108,"[62200140, 4557438, 62156405, 80333174, 38309702, 59671387, 66164952, 37757410, 16294767, 44878788, 38747542, 42598420, 18663560, 10517020, 6211961, 7298112, 13225945, 8043288, 10920382, 18219005, 32363526, 17566661, 62623184, 4763573, 20853216, 62543608, 62501359, 14739479, 20976235, 18669105, 5377188, 32313561, 18263576, 7438775, 166464249, 12867643, 62398150, 21049052, 1809293, 7089159, 7441750, 4137138, 62810060, 62521088, 21076498, 168069660, 166958817, 65122773, 62422194, 51220241, 9191633, 62095026, 12106659, 62354250, 40132861, 62665934, 40686252, 9505311, 62694751, 63423769, 59732024, 62659413, 62883525, 63133221, 61865720, 20629788, 29371220, 57584638, 5711881, 63452797, 63468226, 51679236, 62545344, 8187004, 63078357, 166608363, 28240379, 58453185, 63132923, 77351967, 28669498, 28360306, 15700990, 17587826, 66373416, 6569127, 28273397, 20710384, 67119969, 63363572, 37969849, 36311677, 63129880, 58930696, 40726507, 27931540, 47513944, 66602623, 63374013, 56215621, ...]",membrane protein complex
GO:1902494,0.271,1.614,0.003,0.013,147,111,"[22932922, 18105284, 66137094, 23090099, 15891262, 29629717, 74428367, 62396051, 55309142, 53458440, 52973857, 58407891, 59158322, 60953143, 62674288, 41397517, 62814318, 63525023, 39288217, 36913304, 62942169, 62816710, 12856286, 39325977, 51290647, 49198353, 62833000, 21228956, 62245036, 61651156, 42963082, 67147359, 36803430, 40686252, 62883525, 62396515, 63133221, 61865720, 58064839, 20629788, 29371220, 57584638, 66309200, 5711881, 63452797, 63468226, 51679236, 62545344, 63652778, 8187004, 63078357, 166608363, 28240379, 58453185, 63132923, 77351967, 28669498, 28360306, 15700990, 17587826, 66373416, 6569127, 28273397, 20710384, 67119969, 63363572, 37969849, 36311677, 63129880, 58930696, 66525851, 66783164, 40726507, 27

In [25]:
cc_control

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0042995,-0.623,-3.391,0.000,0.000,199,189,"[63669394, 55916126, 56351287, 50235670, 51356855, 40898493, 63402024, 62923262, 62013121, 62273092, 46374759, 64855697, 65866265, 64376861, 66088492, 63275290, 62529481, 62610295, 62146864, 62706371, 66772951, 53460814, 62047822, 63452851, 64065097, 63842059, 62543065, 62436418, 63865661, 63245372, 62525835, 64780867, 167602424, 167949258, 167601916, 167560438, 167355590, 167344702, 167246340, 63954167, 167732614, 62682149, 62451969, 57528177, 49587153, 63277926, 40260495, 62408493, 65774791, 62097905, 39317954, 64415346, 67218229, 62560831, 62613597, 62966887, 13571055, 13589494, 166985137, 168130942, 165775316, 51837448, 63306571, 64917247, 43280069, 28438217, 167799665, 62350005, 62838017, 62619244, 3775835, 62698072, 62766323, 62509179, 62510437, 62298774, 62926584, 65968008, 64297496, 62803002, 51263942, 22240606, 166703502, 64593986, 62613856, 62891467, 22255856, 13571057, 166907574, 168077902, 63094536, 63839639, 64099377, 62206100, 56924487, 62565570, 63039819, 63721498, 62769590, 63356547, ...]",cell projection
GO:0044463,-0.630,-3.377,0.000,0.000,174,166,"[63669394, 55916126, 50235670, 62923262, 62013121, 62273092, 46374759, 64855697, 65866265, 64376861, 66088492, 63275290, 62529481, 62610295, 62146864, 62706371, 66772951, 53460814, 62047822, 63452851, 62543065, 62436418, 63865661, 63245372, 62525835, 64780867, 167602424, 167949258, 167601916, 167560438, 167355590, 167344702, 167246340, 63954167, 167732614, 62682149, 62451969, 57528177, 49587153, 63277926, 40260495, 62408493, 65774791, 62097905, 39317954, 64415346, 67218229, 62560831, 62613597, 62966887, 13571055, 13589494, 166985137, 168130942, 51837448, 63306571, 64917247, 43280069, 28438217, 167799665, 62350005, 62838017, 62619244, 3775835, 62698072, 62766323, 62509179, 62510437, 62298774, 62926584, 64297496, 62803002, 22240606, 166703502, 64593986, 62613856, 62891467, 22255856, 13571057, 166907574, 168077902, 63094536, 63839639, 64099377, 62206100, 56924487, 62565570, 63039819, 63721498, 62769590, 63356547, 21809431, 62719918, 62784306, 62303301, 62473024, 166208736, 64272473, 65737380, 62815844, ...]",cell projection part
GO:0016020,-0.575,-3.271,0.000,0.000,394,268,"[34034060, 13234966, 33736016, 60409864, 14425887, 39552245, 21687303, 34041938, 63059959, 62049799, 63188988, 21215010, 62484613, 18752817, 80384152, 32671046, 61607570, 61625366, 63123170, 57798810, 46574529, 33471949, 37225003, 66484337, 58120150, 64649372, 67223674, 21807888, 66122622, 62054139, 56035352, 49582748, 62730692, 42743132, 22604158, 44522527, 46089913, 64694136, 38737848, 62378023, 7497632, 52908245, 62726174, 52808333, 61503918, 62361740, 43266065, 44499302, 64320533, 66402745, 45722164, 52955187, 42300035, 61002121, 65303219, 43424619, 52609475, 59052976, 47037033, 63338002, 42005004, 61712897, 21008359, 21814424, 43082532, 61957888, 36220708, 50420785, 8103948, 52863741, 57209297, 54098061, 61616124, 38857988, 36293304, 51540385, 61632074, 59053239, 61649530, 63844951, 58536749, 55774214, 56581658, 61638635, 39489881, 51821180, 62395108, 44394413, 62212023, 61569703, 52655657, 59055758, 50225241, 57033043, 52061002, 44079911, 38310319, 51323646, 42758340, 42827533, ...]",membrane
GO:1904949,-0.420,-1.808,0.002,0.003,52,47,"[40686252, 62883525, 63133221, 61865720, 20629788, 29371220, 57584638, 5711881, 63452797, 63468226, 51679236, 62545344, 8187004, 63078357, 166608363, 28240379, 58453185, 63132923, 77351967, 28669498, 28360306, 15700990, 17587826, 66373416, 6569127, 28273397, 20710384, 67119969, 63363572, 37969849, 36311677, 63129880, 58930696, 40726507, 27931540, 47513944, 66602623, 63374013, 56215621, 22187175, 40674602, 66462102, 2647227, 62946388, 62874519, 41124596, 10689069]",ATPase complex
GO:1990351,-0.420,-1.803,0.001,0.003,52,47,"[40686252, 62883525, 63133221, 61865720, 20629788, 29371220, 57584638, 5711881, 63452797, 63468226, 51679236, 62545344, 8187004, 